## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-14-22-54-35 +0000,48hills,"BIG WEEK: Honoring MLK, SF Sketchfest, SF Art ...",https://48hills.org/2026/01/big-week-honoring-...
1,2026-01-14-22-53-34 +0000,nypost,LI killer couple who hacked up roommates with ...,https://nypost.com/2026/01/14/us-news/li-kille...
2,2026-01-14-22-52-30 +0000,wapo,Vance’s Greenland meeting ends with ‘fundament...,https://www.washingtonpost.com/national-securi...
3,2026-01-14-22-50-40 +0000,nyt,Renaming Defense Department Could Cost Taxpaye...,https://www.nytimes.com/2026/01/14/us/defense-...
4,2026-01-14-22-48-00 +0000,wsj,Paramount Skydance Names Cinelli CFO Amidst Fi...,https://www.wsj.com/business/media/paramount-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
56,trump,63
60,iran,35
116,new,20
274,ice,17
1158,china,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
97,2026-01-14-20-36-08 +0000,nypost,Trump says Iran has stopped killing protesters...,https://nypost.com/2026/01/14/us-news/trump-sa...,132
110,2026-01-14-20-17-00 +0000,wsj,"Fed governor Stephen Miran, an appointee of Pr...",https://www.wsj.com/economy/central-banking/fe...,130
39,2026-01-14-22-10-00 +0000,wsj,Trump Says Iran Has Stopped Killing Protesters...,https://www.wsj.com/world/middle-east/u-s-evac...,128
172,2026-01-14-17-25-00 +0000,nypost,Iran issues sickening assassination threat aga...,https://nypost.com/2026/01/14/world-news/iran-...,125
6,2026-01-14-22-45-00 +0000,nyt,U.S. Moves Some Personnel From Qatar Air Base ...,https://www.nytimes.com/2026/01/14/us/politics...,123


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
97,132,2026-01-14-20-36-08 +0000,nypost,Trump says Iran has stopped killing protesters...,https://nypost.com/2026/01/14/us-news/trump-sa...
110,67,2026-01-14-20-17-00 +0000,wsj,"Fed governor Stephen Miran, an appointee of Pr...",https://www.wsj.com/economy/central-banking/fe...
375,61,2026-01-14-00-11-20 +0000,wapo,New York governor will push for state lawsuits...,https://www.washingtonpost.com/nation/2026/01/...
212,46,2026-01-14-15-23-05 +0000,nyt,China Announces Record Trade Surplus as Its Ex...,https://www.nytimes.com/2026/01/13/business/ch...
145,45,2026-01-14-18-47-34 +0000,startribune,Renee Good came to Minneapolis seeking a safe ...,https://www.startribune.com/renee-good-found-r...
193,45,2026-01-14-16-36-48 +0000,nypost,Home sales slump dragged through 2025 as mortg...,https://nypost.com/2026/01/14/business/home-sa...
49,41,2026-01-14-21-52-49 +0000,bbc,Denmark warns of 'fundamental disagreement' af...,https://www.bbc.com/news/articles/cx20vzz0g8vo...
150,40,2026-01-14-18-27-54 +0000,latimes,California launches investigation into child p...,https://www.latimes.com/california/story/2026-...
284,37,2026-01-14-11-00-56 +0000,wapo,You don’t need Trump to cap your credit card i...,https://www.washingtonpost.com/business/2026/0...
327,36,2026-01-14-06-09-42 +0000,cbc,At least 29 dead after crane collapses onto pa...,https://www.cbc.ca/news/world/thailand-crane-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
